In [1]:
!pip install black[jupyter] --quiet


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


hello this is for code 👍

In [1]:
import openai
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import requests

# --------------------- CONFIGURATION ---------------------
OPENROUTER_API_KEY = "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"
API_URL = "https://openrouter.ai/api/v1/chat/completions"

# --------------------- FUNCTION: CALL LLAMA 3.3 ---------------------
def call_llama(prompt, temperature=0.7, max_tokens=2048):
    headers = {
        'Authorization': f'Bearer {OPENROUTER_API_KEY}',
        'Content-Type': 'application/json'
    }
    payload = {
        "model": "meta-llama/llama-3-70b-instruct",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    response = requests.post(API_URL, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        return f"Error: {response.status_code}, {response.text}"

# --------------------- UI COMPONENTS ---------------------
topic_input = widgets.Text(
    value='',
    placeholder='Enter Topic (e.g., Photosynthesis)',
    description='Topic:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

level_dropdown = widgets.Dropdown(
    options=['Foundation', 'Intermediate', 'Advanced'],
    value='Foundation',
    description='Level:',
    style={'description_width': 'initial'}
)

submit_button = widgets.Button(description="Generate Questions", button_style='success')
output_area = widgets.Output()

# Pagination Variables
current_page = 0
subtopics_per_page = 5
all_subtopics = []
all_questions = {}
accordion_widgets = []
question_widgets = []

# --------------------- EVENT HANDLERS ---------------------
def on_submit_clicked(b):
    global all_subtopics, all_questions, current_page
    clear_output()
    display(topic_input, level_dropdown, submit_button, output_area)

    topic = topic_input.value
    level = level_dropdown.value

    if not topic:
        with output_area:
            clear_output()
            display(Markdown("**Please enter a Topic.**"))
        return

    with output_area:
        clear_output()
        display(Markdown("**Fetching Subtopics... Please wait...**"))

    subtopics_prompt = f"""
    You are an AI curriculum expert.

    TASK:
    List essential subtopics under the topic "{topic}" for "{level}" level.

    Rules:
    - Only output a numbered list of subtopics.
    - No introductions or explanations.
    """

    subtopics_text = call_llama(subtopics_prompt)
    all_subtopics = []
    for line in subtopics_text.strip().split('\n'):
        if '.' in line:
            subtopic = line.split('.', 1)[1].strip()
            if subtopic:
                all_subtopics.append(subtopic)

    # Generate Questions for Each Subtopic
    with output_area:
        clear_output()
        display(Markdown("**Generating Questions... Please wait...**"))

    all_questions = {}
    for subtopic in all_subtopics:
        questions_prompt = f"""
        You are an AI generating diagnostic questions.

        TASK:
        Generate 3-5 questions for the subtopic "{subtopic}" at "{level}" level.

        Rules:
        - Only output questions.
        - Numbered list format.
        """
        questions_text = call_llama(questions_prompt)
        questions_list = []
        for line in questions_text.strip().split('\n'):
            if '.' in line:
                question = line.split('.', 1)[1].strip()
                if question:
                    questions_list.append(question)
        all_questions[subtopic] = questions_list

    current_page = 0
    show_page(current_page)

def show_page(page_num):
    global accordion_widgets, question_widgets
    with output_area:
        clear_output()
        display(Markdown(f"### Page {page_num + 1}"))

        start_idx = page_num * subtopics_per_page
        end_idx = start_idx + subtopics_per_page
        page_subtopics = all_subtopics[start_idx:end_idx]

        accordion_items = []
        question_widgets = []

        for subtopic in page_subtopics:
            subtopic_questions = []
            for q in all_questions[subtopic]:
                q_label = widgets.Label(value=f"{q}")
                q_text = widgets.Textarea(placeholder='Your answer here...', layout=widgets.Layout(width='100%', height='60px'))
                question_widgets.append((q, q_text))
                subtopic_questions.append(widgets.VBox([q_label, q_text]))
            accordion_items.append(widgets.VBox(subtopic_questions))

        accordion = widgets.Accordion(children=accordion_items)
        for i, subtopic in enumerate(page_subtopics):
            accordion.set_title(i, subtopic)

        display(accordion)

        # Navigation Buttons
        nav_buttons = []
        if page_num > 0:
            prev_btn = widgets.Button(description="Previous Page", button_style='warning')
            prev_btn.on_click(on_prev_page)
            nav_buttons.append(prev_btn)
        if end_idx < len(all_subtopics):
            next_btn = widgets.Button(description="Next Page", button_style='info')
            next_btn.on_click(on_next_page)
            nav_buttons.append(next_btn)

        submit_btn = widgets.Button(description="Submit Answers", button_style='primary')
        submit_btn.on_click(on_answers_submit)
        nav_buttons.append(submit_btn)

        display(widgets.HBox(nav_buttons))

def on_prev_page(b):
    global current_page
    if current_page > 0:
        current_page -= 1
        show_page(current_page)

def on_next_page(b):
    global current_page
    if (current_page + 1) * subtopics_per_page < len(all_subtopics):
        current_page += 1
        show_page(current_page)

def on_answers_submit(b):
    with output_area:
        clear_output()
        display(Markdown("**Analyzing Your Answers... Please wait...**"))

    collected_answers = ""
    for q, widget in question_widgets:
        answer = widget.value.strip()
        collected_answers += f"Question: {q}\nAnswer: {answer}\n\n"

    analysis_prompt = f"""
    Analyze the following answers for correctness. For each:
    1. Mention if the answer is Correct, Partially Correct, or Incorrect.
    2. Explain why.
    3. Suggest resources (YouTube videos/websites) to improve understanding.

    Answers:
    {collected_answers}
    """

    feedback = call_llama(analysis_prompt)

    with output_area:
        clear_output()
        display(Markdown("### Feedback & Learning Resources:"))
        display(Markdown(feedback))

# --------------------- SETUP UI ---------------------
submit_button.on_click(on_submit_clicked)
display(topic_input, level_dropdown, submit_button, output_area)


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/LLAMA3_Subtopics_Pagination_Accordion.ipynb'

In [ ]:
import openai
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import requests

# --------------------- CONFIGURATION ---------------------
OPENROUTER_API_KEY = "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"
API_URL = "https://openrouter.ai/api/v1/chat/completions"

# --------------------- FUNCTION: CALL LLAMA 3.3 ---------------------
def call_llama(prompt, temperature=0.7, max_tokens=2048):
    headers = {
        'Authorization': f'Bearer {OPENROUTER_API_KEY}',
        'Content-Type': 'application/json'
    }
    payload = {
        "model": "meta-llama/llama-3-70b-instruct",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    response = requests.post(API_URL, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        return f"Error: {response.status_code}, {response.text}"

# --------------------- UI COMPONENTS ---------------------
topic_input = widgets.Text(
    value='',
    placeholder='Enter Topic (e.g., Photosynthesis)',
    description='Topic:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

level_dropdown = widgets.Dropdown(
    options=['Foundation', 'Intermediate', 'Advanced'],
    value='Foundation',
    description='Level:',
    style={'description_width': 'initial'}
)

submit_button = widgets.Button(description="Generate Questions", button_style='success')
output_area = widgets.Output()

# Pagination Variables
current_page = 0
subtopics_per_page = 5
all_subtopics = []
all_questions = {}
question_widgets = []

# --------------------- EVENT HANDLERS ---------------------
def on_submit_clicked(b):
    global all_subtopics, all_questions, current_page
    clear_output()
    display(topic_input, level_dropdown, submit_button, output_area)

    topic = topic_input.value
    level = level_dropdown.value

    if not topic:
        with output_area:
            clear_output()
            display(Markdown("**Please enter a Topic.**"))
        return

    with output_area:
        clear_output()
        display(Markdown("**Fetching Subtopics... Please wait...**"))

    subtopics_prompt = f"""
    You are an AI curriculum expert.

    TASK:
    List essential subtopics under the topic "{topic}" for "{level}" level.

    Rules:
    - Only output a numbered list of subtopics.
    - No introductions or explanations.
    """

    subtopics_text = call_llama(subtopics_prompt)
    all_subtopics = []
    for line in subtopics_text.strip().split('\n'):
        if '.' in line:
            subtopic = line.split('.', 1)[1].strip()
            if subtopic:
                all_subtopics.append(subtopic)

    with output_area:
        clear_output()
        display(Markdown("**Generating Questions... Please wait...**"))

    all_questions = {}
    for subtopic in all_subtopics:
        questions_prompt = f"""
        You are an AI generating diagnostic questions.

        TASK:
        Generate 3-5 questions for the subtopic "{subtopic}" at "{level}" level.

        Rules:
        - Only output questions.
        - Numbered list format.
        """
        questions_text = call_llama(questions_prompt)
        questions_list = []
        for line in questions_text.strip().split('\n'):
            if '.' in line:
                question = line.split('.', 1)[1].strip()
                if question:
                    questions_list.append(question)
        all_questions[subtopic] = questions_list

    current_page = 0
    show_page(current_page)

def show_page(page_num):
    global question_widgets, current_page
    with output_area:
        clear_output()
        display(Markdown(f"### Page {page_num + 1}"))

        start_idx = page_num * subtopics_per_page
        end_idx = min(start_idx + subtopics_per_page, len(all_subtopics))
        page_subtopics = all_subtopics[start_idx:end_idx]

        accordion_items = []
        question_widgets = []

        for subtopic in page_subtopics:
            subtopic_questions = []
            for q in all_questions[subtopic]:
                q_label = widgets.Label(value=f"{q}")
                q_text = widgets.Textarea(placeholder='Your answer here...', layout=widgets.Layout(width='100%', height='60px'))
                question_widgets.append((q, q_text))
                subtopic_questions.append(widgets.VBox([q_label, q_text]))
            accordion_items.append(widgets.VBox(subtopic_questions))

        accordion = widgets.Accordion(children=accordion_items)
        for i, subtopic in enumerate(page_subtopics):
            accordion.set_title(i, subtopic)

        display(accordion)

        # Navigation & Submit Controls
        nav_buttons = []

        prev_btn = widgets.Button(description="Previous Page", button_style='warning', layout=widgets.Layout(width='150px'))
        next_btn = widgets.Button(description="Next Page", button_style='info', layout=widgets.Layout(width='150px'))
        submit_btn = widgets.Button(description="Submit All Answers", button_style='success', layout=widgets.Layout(width='200px'))

        prev_btn.on_click(on_prev_page)
        next_btn.on_click(on_next_page)
        submit_btn.on_click(on_answers_submit)

        if page_num > 0:
            nav_buttons.append(prev_btn)
        if end_idx < len(all_subtopics):
            nav_buttons.append(next_btn)

        # Submit button is always shown
        nav_buttons.append(submit_btn)

        display(widgets.HBox(nav_buttons))

def on_prev_page(b):
    global current_page
    if current_page > 0:
        current_page -= 1
        show_page(current_page)

def on_next_page(b):
    global current_page
    if (current_page + 1) * subtopics_per_page < len(all_subtopics):
        current_page += 1
        show_page(current_page)

def on_answers_submit(b):
    with output_area:
        clear_output()
        display(Markdown("**Analyzing Your Answers... Please wait...**"))

    collected_answers = ""
    for q, widget in question_widgets:
        answer = widget.value.strip()
        collected_answers += f"Question: {q}\nAnswer: {answer}\n\n"

    analysis_prompt = f"""
    Analyze the following answers for correctness. For each:
    1. Mention if the answer is Correct, Partially Correct, or Incorrect.
    2. Explain why.
    3. Suggest resources (YouTube videos/websites) to improve understanding.

    Answers:
    {collected_answers}
    """

    feedback = call_llama(analysis_prompt)

    with output_area:
        clear_output()
        display(Markdown("### Feedback & Learning Resources:"))
        display(Markdown(feedback))

# --------------------- SETUP UI ---------------------
submit_button.on_click(on_submit_clicked)
display(topic_input, level_dropdown, submit_button, output_area)


Text(value='finintial management group 2 2025 ca ', description='Topic:', layout=Layout(width='50%'), placehol…

Dropdown(description='Level:', index=1, options=('Foundation', 'Intermediate', 'Advanced'), style=DescriptionS…

Button(button_style='success', description='Generate Questions', style=ButtonStyle())

Output()